(Übung_Auswertung)=
# 🚀 Übung: Auswerten von CSV-Dateien in R

In der vorangegangenen Übung wurden Sie mit Ansätzen zur Organisation und Strukturierung von CSV-Dateien vertraut gemacht. Der folgende Abschnitt baut auf dem Code der letzten Übung auf und thematisiert Auswertungsmöglichkeiten von CSV-Dateien in R mit Fokus auf Untergruppierungen und Filterfunktionen.

````{admonition} Den Code für die Übung: Arbeiten mit CSV-Dateien in R finden Sie hier: 
:class: hinweis, dropdown
```
#TidyVerse Package Installation
install.packages("tidyverse")
library(tidyverse)

#Daten einlesen
data_csv_clean <- read.csv2("Data/21341-0001_F_2020.csv", header = FALSE, encoding = "latin1")

#Ergebnisse ansehen
head(data_csv_clean)

#Umlaute entfernen
data_csv_clean$V1 <- str_replace_all(data_csv_clean$V1, c("ä" = "ae", "ö" = "oe", "ü" ="ue", "ß" ="ss"))
data_csv_clean$V2 <- str_replace_all(data_csv_clean$V2, c("ä" = "ae", "ö" = "oe", "ü" ="ue", "ß" ="ss"))

#Ausgewählte Ergebnisse ansehen
show(data_csv_clean[8:20,1:3])

#Tabelle unterteilen
Metadaten <- data_csv_clean[c(1:6, 87:88), 1]
Tabellendaten <- data_csv_clean[8:85, 1:3]

#Spaltenüberschriften setzen
colnames(Tabellendaten) <- c("Angestelltenverhaeltnis", "Geschlecht", "Angestelltenzahl_2020")

#Nummerierung neu setzen
row.names(Tabellendaten) <- 1:78

#Ergebnis ansehen
head(Tabellendaten) 
```
````

In [1]:
install.packages("tidyverse")
library(tidyverse)

data_csv_clean <- read.csv2("Data/21341-0001_F_2020.csv", header = FALSE, encoding = "latin1")

data_csv_clean$V1 <- str_replace_all(data_csv_clean$V1, c("ä" = "ae", "ö" = "oe", "ü" ="ue", "ß" ="ss"))
data_csv_clean$V2 <- str_replace_all(data_csv_clean$V2, c("ä" = "ae", "ö" = "oe", "ü" ="ue", "ß" ="ss"))

Metadaten <- data_csv_clean[c(1:6, 87:88), 1]
Tabellendaten <- data_csv_clean[8:85, 1:3]

colnames(Tabellendaten) <- c("Angestelltenverhaeltnis", "Geschlecht", "Angestelltenzahl_2020")

row.names(Tabellendaten) <- 1:78

Installiere Paket nach ‘/opt/homebrew/lib/R/4.5/site-library’
(da ‘lib’ nicht spezifiziert)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Variablenklassen bestimmen 

Damit Sie Ihre Daten adäquat auswerten können, müssen Sie sich über die
Struktur und Klasse der Variablen bewusst sein.  

**Die wichtigsten Variablenklassen im Überblick:**  

-   `integer`: Ganze Zahlen.  
    Beispiel: 1, 2, 5.

-   `numeric`: Zahl mit Kommastellen.  
    Beispiel: 1.99, 2.33, 5.00.

-   `character`: Zeichenketten bzw. Strings.  
    Beispiel: “Hallo”, “Apfel”, “Hannah”.

-   `factor`: Kategoriale Variablen.  
    Beispiel: “schlecht”, “gut”, “sehr gut”

Die Variablenklasse ist ausschlaggebend dafür, wie Sie die
Variablendaten auswerten können. Beispielsweise können Sie mit einer
`numeric`-Variable rechnen, mit einer `character`-Variable jedoch
nicht. 

`````{margin}
````{admonition} Tipp für RStudio
:class: hinweis
Im *Environment* Fenster können Sie die Variablenklassen ebenfalls schnell einsehen.


```{figure} _images/R_Environment_Variablenklassen.png
---
name: r-environment
alt: Ein Screenshot, der den Ausschnitt des Environment-Fensters von R-Studio zeigt.
---
Environment-Fenster in RStudio.
```
````
`````

**Variablenklassen bestimmen im Code:**  
Der Befehl `class()` gibt uns die jeweilige Klasse des Objektes wieder:  

In [2]:
class(Tabellendaten$Angestelltenverhaeltnis) #sollte als character oder factor gelesen werden

[1] "character"

In [3]:
class(Tabellendaten$Geschlecht) #sollte als character oder factor gelesen werden

[1] "character"

In [4]:
class(Tabellendaten$Angestelltenzahl_2020) #sollte als numerisch oder integer gelesen werden

[1] "character"

Die Variablenklassen für “Angestelltenverhaeltnis” und “Geschlecht” sind
passend. Die Variable “Angestelltenzahl_2020” wird jedoch auch als
`character` interpretiert. Dies muss manuell geändert werden, da sonst
bestimmte Auswertungen nicht möglich sind und Fehler entstehen können.  

**Fallbeispiel:** Betrachten Sie die Funktion `max()`, welche den
Maximalwert in der jeweiligen Spalte zurückgeben soll. Hinweis:
`na.rm = TRUE` muss hinzugefügt werden, damit die nicht vorhandenen
Werte nicht berücksichtigt werden, ansonsten kommt es zu einem Error.

In [5]:
max(Tabellendaten$Angestelltenzahl_2020, na.rm = TRUE)

[1] "99730"

Wenn Sie jedoch einen Blick in die Tabelle werfen, sehen Sie schnell,
dass es mehrere Angestelltenzahlen über 100.000 gibt. Die ausgegebene
Zahl kann also nicht stimmen!

**Wie kann es zu solch einem Fehler kommen?**  
Da die Variable “Angestelltenzahl\_2020” als `character` interpretiert
wird, ändert sich auch die Bedeutung der Funktion `max()`. Die Funkion
`max()` gibt nun nicht mehr den Maximalwert zurück, sondern den
niedrigsten Platz in alphabetischer Sortierung. Nur wenn wenn die Werte
in der Tabelle auch als Zahlen (also numerisch/integer) von **R**
gelesen werden, gibt die Funktion `max()` den Maximalwert der Spalte
zurück.

(Variablenklasse)=
## Variablenklasse ändern 

Hierfür überspeichern Sie die Daten der jeweiligen Spalte mit den exakt
selben Daten, nur das Sie durch den Befehl `as.integer` die Klasse der
Variable ändern. Wenn Sie die Klasse in numerisch ändern wollen, würden
Sie `as.numeric` verwenden.

In [6]:
Tabellendaten$Angestelltenzahl_2020 <- as.integer(Tabellendaten$Angestelltenzahl_2020)

Warning message:
“NAs durch Umwandlung erzeugt”


Überprüfen Sie gegebenenfalls erneut die Klasse:

In [7]:
class(Tabellendaten$Angestelltenzahl_2020)

[1] "integer"

Wenn Sie nun noch einmal die Beispielfunktion `max()` heranziehen, dann
bekommen Sie jetzt durch neuer Klassenzugehörigkeit der Variable den
Maximalwert angezeigt:

In [8]:
max(Tabellendaten$Angestelltenzahl_2020, na.rm = TRUE)

[1] 759065

```{admonition} Wichtig!
:class: keypoint
Überprüfen Sie immer die Art der Variablenklasse, welche **R** Ihren Daten zuweist und ändern Sie dies gegebenenfalls, um Fehler zu vermeiden.
```

## Variablen hinzufügen 

Eine gute **Kategorisierung** Ihrer Daten ist notwendig, um effizient zu
filtern. Hierzu müssen Sie die Struktur der Daten nachvollziehen
können.  
Um die einzelnen Ausprägung der Variable “Angestelltenverhaeltnis” zu
sichten, bietet Ihnen **R** den Befehl `unique()`:

In [9]:
unique(Tabellendaten$Angestelltenverhaeltnis)

[1] "Wissenschaftliches und kuenstlerisches Personal"     
 [2] "  Hauptberufl. wissenschaftl. u. kuenstler. Personal"
 [3] "    Professoren"                                     
 [4] "    Dozenten und Assistenten"                        
 [5] "    Wissenschaftliche und kuenstlerische Mitarbeiter"
 [6] "    Lehrkraefte fuer besondere Aufgaben"             
 [7] "  Nebenberufl. wissenschaftl. u. kuenstler. Personal"
 [8] "    Gastprofessoren, Emeriti"                        
 [9] "    Lehrbeauftragte"                                 
[10] "    Wissenschaftliche Hilfskraefte"                  
[11] "Verwaltungs-, technisches und sonstiges Personal"    
[12] "  Hauptberufl. Verwaltungs-, techn. u.sonst.Personal"
[13] "    Verwaltungspersonal"                             
[14] "    Bibliothekspersonal"                             
[15] "    Technisches Personal"                            
[16] "    Sonstiges Personal"                              
[17] "    Pflegepersonal"                                  
[18] "    Arbeiter"                                        
[19] "    Auszubildende"                                   
[20] "    Praktikanten"                                    
[21] "    Sonstiges Personal fuer Lehre und Forschung"     
[22] "    Leitungs- und Verwaltungspersonal"               
[23] "  Nebenberufl. Verwaltungs-, techn. u.sonst.Personal"
[24] "    Sonstige Hilfskraefte"                           
[25] "    Zeitweilig Beschaeftigte"                        
[26] "Insgesamt"

Wie Sie erkennen können, ist die Variable “Angestelltenverhaeltnis”
recht unübersichtlich, da sie verschiedene Unterkategorien enthält. Bei
Betrachtung der einzelnen Positionen ist jedoch nicht erkennbar, zu
welcher Überkategorie die Position gehört.

**Bei genauerer Betrachtung der Tabelle fällt auf, dass es drei Ebenen
gibt:**

1.  Ebene: “Wissenschaftliches und kuenstlerisches Personal” und
    “Verwaltungs-, technisches und sonstiges Personal”  

2.  Ebene: “Hauptberuflich” und “Nebenberuflich”

3.  Ebene: Die einzelnen Positionen der Berufsbezeichnungen z.B.
    “Professoren”, “Dozenten und Assistenten”, etc.

Für eine übersichtlichere Struktur in der Tabelle kann für die ersten
zwei Ebenen jeweils eine neue Variable erstellt werden. Hierdurch werden
zwei neue Spalten zur Tabelle hinzugefügt und Sie können dann bei
Betrachtung einer einzelnen Zeile alle wichtigen Informationen
herauslesen, was derzeit nicht möglich ist.  
  
**Neue Variablen erstellen**:  
Da die Struktur es nicht erlaubt, die neuen Variablen durch Referenz auf
die bereits vorhandenen Variablen zu erzeugen, müssen die neuen
Variablen manuell strukturiert werden.  

Erzeugen Sie hierfür zunächst neue Variablen und setzen Sie den Inhalt
erst mal auf unbekannt (NA).

In [10]:
#1.Ebene: Neue Variable "Personalkategorie"
Tabellendaten$Personalkategorie <- c(NA)

In [11]:
#2.Ebene: Neue Variable "Art der Anstellung"
Tabellendaten$Art_der_Anstellung <- c(NA)

Der Spalteninhalt muss nun manuell eingetragen werden.  
Achtung: Diese Weise ist leider recht fehleranfällig, ist jedoch durch
den Aufbau der Datei, ohne komplexere Funktionen zu benutzen, nicht
anders möglich.

```{admonition} Wichtig!
:class: caution
Wenn Sie Daten generieren, strukturieren Sie diese immer gut. Dies erleichtert Ihnen selbst, aber auch allen anderen, die Ihre Daten verwenden, das Arbeiten! Eine gute Struktur von Daten liefert der Tidy-Data Ansatz (s. Abschnitt [Datenstruktur](/Markdown/6_1_Datenstruktur.md)).
```

**Einpflegen des Variableninhalts**

In [12]:
#1.Ebene
Tabellendaten$Personalkategorie[76:78] <- "Insgesamt"
Tabellendaten$Personalkategorie[1:30] <- "Wissenschaftliches und künstlerisches Personal"
Tabellendaten$Personalkategorie[31:75] <- "Verwaltungs-, technisches und sonstiges Personal"

In [13]:
#2.Ebene
Tabellendaten$Art_der_Anstellung[c(1:3, 31:33, 76:78)] <- "Insgesamt"
Tabellendaten$Art_der_Anstellung[c(4:18,34:66)] <- "Hauptberuflich"
Tabellendaten$Art_der_Anstellung[c(19:30,67:75)] <- "Nebenberuflich"

Ergebnis ansehen:

In [14]:
head(Tabellendaten, n=78)

,Angestelltenverhaeltnis,Geschlecht,Angestelltenzahl_2020,Personalkategorie,Art_der_Anstellung
,<chr>,<chr>,<int>,<chr>,<chr>
1,Wissenschaftliches und kuenstlerisches Personal,maennlich,247720,Wissenschaftliches und künstlerisches Personal,Insgesamt
2,Wissenschaftliches und kuenstlerisches Personal,weiblich,167112,Wissenschaftliches und künstlerisches Personal,Insgesamt
3,Wissenschaftliches und kuenstlerisches Personal,Insgesamt,414832,Wissenschaftliches und künstlerisches Personal,Insgesamt
4,Hauptberufl. wissenschaftl. u. kuenstler. Personal,maennlich,159567,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
5,Hauptberufl. wissenschaftl. u. kuenstler. Personal,weiblich,109708,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
6,Hauptberufl. wissenschaftl. u. kuenstler. Personal,Insgesamt,269275,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
7,Professoren,maennlich,36344,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
8,Professoren,weiblich,12949,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
9,Professoren,Insgesamt,49293,Wissenschaftliches und künstlerisches Personal,Hauptberuflich


## Unterkategorisieren 

Eine Unterkategorisierung (auch Sub-Sampling gennant) bietet sich auch
insbesondere für die Variable “Geschlecht” an.

In [15]:
unique(Tabellendaten$Geschlecht)

[1] "maennlich" "weiblich"  "Insgesamt"

Durch das Aufführen der Variablenausprägung `"Insgesamt"` wird die
Tabelle recht unübersichtlich. Es empfiehlt sich das sub-samplen der
Tabelle nach den verschiedenen Geschlechtsausprägungen:

In [16]:
#Geschlecht
Tabelle_maennlich <- subset(Tabellendaten, Geschlecht == "maennlich")
Tabelle_weiblich <- subset(Tabellendaten, Geschlecht == "weiblich")
Tabelle_Insgesamt <- subset(Tabellendaten, Geschlecht == "Insgesamt")

In [17]:
#Ergbnis ansehen 
head(Tabelle_Insgesamt)

,Angestelltenverhaeltnis,Geschlecht,Angestelltenzahl_2020,Personalkategorie,Art_der_Anstellung
,<chr>,<chr>,<int>,<chr>,<chr>
3,Wissenschaftliches und kuenstlerisches Personal,Insgesamt,414832,Wissenschaftliches und künstlerisches Personal,Insgesamt
6,Hauptberufl. wissenschaftl. u. kuenstler. Personal,Insgesamt,269275,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
9,Professoren,Insgesamt,49293,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
12,Dozenten und Assistenten,Insgesamt,3728,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
15,Wissenschaftliche und kuenstlerische Mitarbeiter,Insgesamt,205387,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
18,Lehrkraefte fuer besondere Aufgaben,Insgesamt,10867,Wissenschaftliches und künstlerisches Personal,Hauptberuflich


Wenn Sie nun z.B. aus der *Tabelle\_Insgesamt* die zweite Spalte entfernen
möchten, da diese redundant ist, können Sie die Spalte aus dem Subsample
entfernen:

In [18]:
Tabelle_Insgesamt <- subset(Tabelle_Insgesamt, select= c(-Geschlecht))

In [19]:
head(Tabelle_Insgesamt)

,Angestelltenverhaeltnis,Angestelltenzahl_2020,Personalkategorie,Art_der_Anstellung
,<chr>,<int>,<chr>,<chr>
3,Wissenschaftliches und kuenstlerisches Personal,414832,Wissenschaftliches und künstlerisches Personal,Insgesamt
6,Hauptberufl. wissenschaftl. u. kuenstler. Personal,269275,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
9,Professoren,49293,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
12,Dozenten und Assistenten,3728,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
15,Wissenschaftliche und kuenstlerische Mitarbeiter,205387,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
18,Lehrkraefte fuer besondere Aufgaben,10867,Wissenschaftliches und künstlerisches Personal,Hauptberuflich


## Tabellendaten filtern 

Alternativ zum `subset`-Befehl kann mit dem Befehl `filter` gearbeitet
werden.  
Damit können Sie Ihre Tabelle filtern und sich gezielt Inhalte anzeigen lassen.  
  
Hierzu können Sie verschiedene Operatoren und Funktionen nutzen:  
  


**Mathematische Operatoren:**

-   `==` (Ist gleich)
-   `!=` (Ist nicht gleich)
-   `<` (Kleiner als)
-   `<=` (Kleiner-Gleich)
-   `>` (Größer als)
-   `>=` (Größer-Gleich)

Beispiel:

In [20]:
filter(Tabelle_Insgesamt, Tabelle_Insgesamt$Angestelltenzahl_2020 <= 5000)

Angestelltenverhaeltnis,Angestelltenzahl_2020,Personalkategorie,Art_der_Anstellung
<chr>,<int>,<chr>,<chr>
Dozenten und Assistenten,3728,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
"Gastprofessoren, Emeriti",1793,Wissenschaftliches und künstlerisches Personal,Nebenberuflich
Praktikanten,2061,"Verwaltungs-, technisches und sonstiges Personal",Hauptberuflich


**Logische Operatoren:**

-   `!` (logisches NICHT)
-   `&` (logisches UND)
-   `|` (logisches ODER)

Beispiel:

In [21]:
filter(Tabellendaten, Tabellendaten$Personalkategorie == "Wissenschaftliches und künstlerisches Personal" & Tabellendaten$Geschlecht == "weiblich" )

Angestelltenverhaeltnis,Geschlecht,Angestelltenzahl_2020,Personalkategorie,Art_der_Anstellung
<chr>,<chr>,<int>,<chr>,<chr>
Wissenschaftliches und kuenstlerisches Personal,weiblich,167112,Wissenschaftliches und künstlerisches Personal,Insgesamt
Hauptberufl. wissenschaftl. u. kuenstler. Personal,weiblich,109708,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
Professoren,weiblich,12949,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
Dozenten und Assistenten,weiblich,1546,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
Wissenschaftliche und kuenstlerische Mitarbeiter,weiblich,89383,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
Lehrkraefte fuer besondere Aufgaben,weiblich,5830,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
Nebenberufl. wissenschaftl. u. kuenstler. Personal,weiblich,57404,Wissenschaftliches und künstlerisches Personal,Nebenberuflich
"Gastprofessoren, Emeriti",weiblich,259,Wissenschaftliches und künstlerisches Personal,Nebenberuflich
Lehrbeauftragte,weiblich,35238,Wissenschaftliches und künstlerisches Personal,Nebenberuflich


**Verwenden von Funktionen:**

Beispiel: Ausgabe des kleinsten Wertes mittels der `min()`-Funktion

In [22]:
filter(Tabellendaten, Tabellendaten$Angestelltenzahl_2020 == min(Tabellendaten$Angestelltenzahl_2020, na.rm = TRUE))

Angestelltenverhaeltnis,Geschlecht,Angestelltenzahl_2020,Personalkategorie,Art_der_Anstellung
<chr>,<chr>,<int>,<chr>,<chr>
"Gastprofessoren, Emeriti",weiblich,259,Wissenschaftliches und künstlerisches Personal,Nebenberuflich


**Mögliche Probleme?**

Wenn mit der Variable `Angestelltenverhaeltnis` gefiltert werden soll, gibt **R** folgenden Output:

In [23]:
#Beispiel: Filtern nach "Professoren"
filter(Tabellendaten, Tabellendaten$Angestelltenverhaeltnis == "Professoren")

Angestelltenverhaeltnis,Geschlecht,Angestelltenzahl_2020,Personalkategorie,Art_der_Anstellung
<chr>,<chr>,<int>,<chr>,<chr>


Wenn Sie jedoch in den Datensatz schauen, sehen Sie, dass Professoren unter `Angestelltenverhaeltnis` mit aufgelistet werden. 

**Warum werden "Professoren" nicht gefunden?**

Das Problem besteht darin, dass die Ausgangsdatei Leerzeichen vor dem Wort Professoren (und auch vor anderen Bezeichnungen von Angestelltenverhältnissen) enthält.

Mit dem Befehl `trimws()` können Sie die vorgestellten Leerzeichen vor den Bezeichnungen entfernen:

In [24]:
Tabellendaten$Angestelltenverhaeltnis <- trimws(Tabellendaten$Angestelltenverhaeltnis)

Führen Sie nun erneut die Filtersuche durch und Sie erhalten das korrekte Ergebnis:

In [25]:
filter(Tabellendaten, Tabellendaten$Angestelltenverhaeltnis == "Professoren")

Angestelltenverhaeltnis,Geschlecht,Angestelltenzahl_2020,Personalkategorie,Art_der_Anstellung
<chr>,<chr>,<int>,<chr>,<chr>
Professoren,maennlich,36344,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
Professoren,weiblich,12949,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
Professoren,Insgesamt,49293,Wissenschaftliches und künstlerisches Personal,Hauptberuflich


## Sortieren von Tabellendaten 

Sie können Tabellen mittels des `order()`-Befehls nach belieben
sortieren.  

Beispiel:

In [26]:
Tabelle_Insgesamt_Sortiert <- Tabelle_Insgesamt[order(Tabelle_Insgesamt$Angestelltenzahl_2020, decreasing = TRUE),]

In [27]:
head(Tabelle_Insgesamt_Sortiert)

,Angestelltenverhaeltnis,Angestelltenzahl_2020,Personalkategorie,Art_der_Anstellung
,<chr>,<int>,<chr>,<chr>
78,Insgesamt,759065,Insgesamt,Insgesamt
3,Wissenschaftliches und kuenstlerisches Personal,414832,Wissenschaftliches und künstlerisches Personal,Insgesamt
33,"Verwaltungs-, technisches und sonstiges Personal",344233,"Verwaltungs-, technisches und sonstiges Personal",Insgesamt
36,"Hauptberufl. Verwaltungs-, techn. u.sonst.Personal",338666,"Verwaltungs-, technisches und sonstiges Personal",Hauptberuflich
6,Hauptberufl. wissenschaftl. u. kuenstler. Personal,269275,Wissenschaftliches und künstlerisches Personal,Hauptberuflich
15,Wissenschaftliche und kuenstlerische Mitarbeiter,205387,Wissenschaftliches und künstlerisches Personal,Hauptberuflich


````{admonition} Zur Kontrolle, finden Sie den kompletten Code der Übung hier: 
:class: hinweis, dropdown
```
#TidyVerse Package Installation
install.packages("tidyverse")
library(tidyverse)

#Daten einlesen
data_csv_clean <- read.csv2("Data/21341-0001_F_2020.csv", header = FALSE, encoding = "latin1")

#Ergebnisse ansehen
head(data_csv_clean)

#Umlaute entfernen
data_csv_clean$V1 <- str_replace_all(data_csv_clean$V1, c("ä" = "ae", "ö" = "oe", "ü" ="ue", "ß" ="ss"))
data_csv_clean$V2 <- str_replace_all(data_csv_clean$V2, c("ä" = "ae", "ö" = "oe", "ü" ="ue", "ß" ="ss"))

#Ausgewählte Ergebnisse ansehen
show(data_csv_clean[8:20,1:3])

#Tabelle unterteilen
Metadaten <- data_csv_clean[c(1:6, 87:88), 1]
Tabellendaten <- data_csv_clean[8:85, 1:3]

#Spaltenüberschriften setzen
colnames(Tabellendaten) <- c("Angestelltenverhaeltnis", "Geschlecht", "Angestelltenzahl_2020")

#Nummerierung neu setzen
row.names(Tabellendaten) <- 1:78

#Ergebnis ansehen
head(Tabellendaten) 

#Variablenklassen bestimmen
class(Tabellendaten$Angestelltenverhaeltnis) #sollte als character oder factor gelesen werden
class(Tabellendaten$Geschlecht) #sollte als character oder factor gelesen werden
class(Tabellendaten$Angestelltenzahl_2020) #sollte als numerisch oder integer gelesen werden

#Variablenklassen ändern
Tabellendaten$Angestelltenzahl_2020 <- as.integer(Tabellendaten$Angestelltenzahl_2020)

#Maximalwert anzeigen lassen
max(Tabellendaten$Angestelltenzahl_2020, na.rm = TRUE)

#Variablen sichten
unique(Tabellendaten$Angestelltenverhaeltnis)

#Neue Variablen erstellen
#1.Ebene: Neue Variable "Personalkategorie"
Tabellendaten$Personalkategorie <- c(NA)
#2.Ebene: Neue Variable "Art der Anstellung"
Tabellendaten$Art_der_Anstellung <- c(NA)

#Einpflegen des Variableninhalts
#1.Ebene
Tabellendaten$Personalkategorie[76:78] <- "Insgesamt"
Tabellendaten$Personalkategorie[1:30] <- "Wissenschaftliches und künstlerisches Personal"
Tabellendaten$Personalkategorie[31:75] <- "Verwaltungs-, technisches und sonstiges Personal"
#2.Ebene
Tabellendaten$Art_der_Anstellung[c(1:3, 31:33, 76:78)] <- "Insgesamt"
Tabellendaten$Art_der_Anstellung[c(4:18,34:66)] <- "Hauptberuflich"
Tabellendaten$Art_der_Anstellung[c(19:30,67:75)] <- "Nebenberuflich"

#Unterkategorisieren
#Geschlecht
Tabelle_maennlich <- subset(Tabellendaten, Geschlecht == "maennlich")
Tabelle_weiblich <- subset(Tabellendaten, Geschlecht == "weiblich")
Tabelle_Insgesamt <- subset(Tabellendaten, Geschlecht == "Insgesamt")

#Redundante Tabelle entfernen
Tabelle_Insgesamt <- subset(Tabelle_Insgesamt, select= c(-Geschlecht))

#Tabellendaten filtern
#Nach Angestelltenzahl unter 5000 filtern
filter(Tabelle_Insgesamt, Tabelle_Insgesamt$Angestelltenzahl_2020 <= 5000)
#Nach weiblichen Wiss.& Künstl.Personal filtern
filter(Tabellendaten, Tabellendaten$Personalkategorie == "Wissenschaftliches und künstlerisches Personal" & Tabellendaten$Geschlecht == "weiblich" )
#Filtern nach niedrigstem Wert
filter(Tabellendaten, Tabellendaten$Angestelltenzahl_2020 == min(Tabellendaten$Angestelltenzahl_2020, na.rm = TRUE))

#Fehlerbehebung Leerzeichen entfernen
Tabellendaten$Angestelltenverhaeltnis <- trimws(Tabellendaten$Angestelltenverhaeltnis)

#Filter nach 'Professoren'
filter(Tabellendaten, Tabellendaten$Angestelltenverhaeltnis == "Professoren")

#Sortieren von Tabellendaten
Tabelle_Insgesamt_Sortiert <- Tabelle_Insgesamt[order(Tabelle_Insgesamt$Angestelltenzahl_2020, decreasing = TRUE),]

#Ergebnis ansehen
head(Tabelle_Insgesamt_Sortiert)
```
````